<script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=TeX-AMS_CHTML"></script>
<script type="text/x-mathjax-config">
MathJax.Hub.Config({
tex2jax: {
inlineMath: [['$','$'], ['\\(','\\)']],
processEscapes: true},
jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
TeX: {
extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
equationNumbers: {
autoNumber: "AMS"
}
}
});
</script>

### Assignment 1: Global alignment

Under the assumption that both input sequences \\(a\\) and \\(b\\) stem from the same origin, a global alignment tries to identify matching parts and the changes needed to transfer one sequence into the other. <br>
The changes are scored and an optimal set of changes is identified, which defines an alignment. <br>
The dynamic programming approach tabularizes optimal subsolutions in matrix \\(D\\), where an entry \\(D_{i,j}\\) represents the best score for aligning the prefixes \\(a_{1..i}\\) with \\(b_{1..j}\\). <br>

<i>To better clarify how global alignment works, take a look here:</i> [Freiburg website](http://rna.informatik.uni-freiburg.de/Teaching/index.jsp?toolName=Needleman-Wunsch)

Write a Python program that given two sequences (passed as first and second argument from command line) and match, mismatch and gap costs (passed as third, fourth, fifth argument from command line):
1. Compute matrix D and output it on the terminal, along with the final alignment score
2. Output the final alignment (if two sequences have more than one alignment with the same score, provide one of them e.g. check website for ‘AACCG’ and ‘AACG’)
3. Check your alignment on Freiburg website

In [45]:
args = ["AACCG", "AACG", 1, -1, -2]

sequence_A = args[0]
sequence_B = args[1]

match    = args[2]
mismatch = args[3]
gap      = args[4]

evaluator = Evaluator(match, mismatch, gap)

SIM, trace, global_alignment_value = evaluator.similarity(sequence_A, sequence_B, "global")

pprint(SIM)
print()

print("Global alignment value: " + str(global_alignment_value))
print()

path, alignment_A, alignment_B = evaluator.global_alignment(trace)

print(alignment_A)
print(alignment_B)

0  	-2	-4	-6	-8
-2 	1 	-1	-3	-5
-4 	-1	2 	0 	-2
-6 	-3	0 	3 	1 
-8 	-5	-2	1 	2 
-10	-7	-4	-1	2 

Global alignment value: 2

AACCG
AA_CG


---
### Assignment 2: Local alignment

A local alignment approach tries to identify the most similar subsequences that maximize the scoring of their matching parts and the changes needed to transfer one subsequence into the other.<br>
The dynamic programming approach tabularizes optimal subsolutions in matrix \\(S\\), where an entry \\(S_{i,j}\\) represents the maximal similarity score for any local alignment of the (sub)prefixes \\(a_{x..i}\\) with \\(b_{y..j}\\), where \\(x,y>0\\) are so far unknown and have to be identified via traceback.<br>

<i>To better clarify how local alignment works, take a look here:</i> [Freiburg website](http://rna.informatik.uni-freiburg.de/Teaching/index.jsp?toolName=Smith-Waterman)<br>

Write a Python program that given two sequences (passed as first and second argument from command line) and match, mismatch and gap cost (passed as third, fourth, fifth argument from command line).
1. Compute matrix \\(D\\) and output it on the terminal, along with the final alignment score (remember that the minimum value you can have in the matrix is 0!!)
2. Output the final alignment (if two sequences have more than one alignment with the same score, provide one of them)
3. Check your alignment on Freiburg website
4. Local alignment has a lot of concepts similar to global alignment, so you can reuse a lot of code you have previously written for global alignment!

In [62]:
args = ["AATCG", "AACG", 1, -1, -2]

sequence_A = args[0]
sequence_B = args[1]

match    = args[2]
mismatch = args[3]
gap      = args[4]

evaluator = Evaluator(match, mismatch, gap)

SIM, trace, local_alignment_value = evaluator.similarity(sequence_A, sequence_B, "local")

pprint(SIM)
print()

idx = [(i,j) for i in range(len(SIM)) for j in range(len(SIM[0])) if SIM[i][j] == local_alignment_value][0]

print("Local alignment value: " + str(local_alignment_value))
print()

path, alignment_A, alignment_B = evaluator.local_alignment(trace, idx)

print(alignment_A)
print(alignment_B)

0	0	0	0	0
0	1	1	0	0
0	1	2	0	0
0	0	0	1	0
0	0	0	1	0
0	0	0	0	2

Local alignment value: 2

AA
AA


In [60]:
class Evaluator:
    def __init__(self, match, mismatch, gap):
        
        self.match      = match
        self.mismatch   = mismatch
        self.gap        = gap
        
    def similarity(self, sequence_A, sequence_B, similarity_type):
        
        self.sequence_A = sequence_A
        self.sequence_B = sequence_B
        
        if similarity_type == "global":
            return self.global_similarity()
        
        if similarity_type == "local":
            return self.local_similarity()
        
    def initialize_matrix(self, m, n, matrix_name = "", similarity_type = ""):
        
        if matrix_name == "SIM":
            return self.initialize_SIM(m, n, similarity_type)
        
        if matrix_name == "trace":
            return self.initialize_trace(m, n)
        
    def global_similarity(self):
        
        m = len(self.sequence_A) + 1
        n = len(self.sequence_B) + 1
        
        SIM   = self.initialize_matrix(m, n, "SIM", "global")
        trace = self.initialize_matrix(m, n, "trace")
        
        for i in range(1, m):
            
            for j in range(1, n):
                
                SIM[i][j], trace[i][j] = self.global_similarity_max(SIM, i, j)
                
        return SIM, trace, SIM[m-1][n-1]
    
    def local_similarity(self):
        
        S = 0
        
        m = len(self.sequence_A) + 1
        n = len(self.sequence_B) + 1
        
        SIM   = self.initialize_matrix(m, n, "SIM", "local")
        trace = self.initialize_matrix(m, n, "trace")
        
        for i in range(1, m):
            
            for j in range(1, n):
                
                SIM[i][j], trace[i][j], S = self.local_similarity_max(SIM, i, j, S)
                
        return SIM, trace, S
        
    def initialize_SIM(self, m, n, similarity_type):
        
        SIM = [["" for y in range(n)] for x in range(m)]
        
        if similarity_type == "global":
            for i in range(m):
                SIM[i][0] = i * self.gap

            for j in range(1, n):
                SIM[0][j] = j * self.gap
                
        if similarity_type == "local":
            for i in range(m):
                SIM[i][0] = 0

            for j in range(1, n):
                SIM[0][j] = 0

        return SIM
    
    def initialize_trace(self, m, n):
        
        trace = [["" for y in range(n)] for x in range(m)]
        
        trace[0][0] = ["done"]
        
        for i in range(1, m):
            trace[i][0] = ["up"]
            
        for j in range(1, n):
            trace[0][j] = ["left"]
            
        return trace
    
    def global_similarity_max(self, SIM, i, j):
        
        current_trace = list()
        
        max_value = max(
                    SIM[i-1][j-1] + self.evaluate_elements(self.sequence_A[i-1], self.sequence_B[j-1]),
                    SIM[i-1][j  ] + self.gap,
                    SIM[i  ][j-1] + self.gap
        ) 
        
        if max_value == SIM[i-1][j-1] + self.evaluate_elements(self.sequence_A[i-1], self.sequence_B[j-1]):
            current_trace.append("diag")
        
        if max_value == SIM[i-1][j  ] + self.gap:
            current_trace.append("up")
            
        if max_value == SIM[i  ][j-1] + self.gap:
            current_trace.append("left")
        
        return max_value, current_trace
    
    def local_similarity_max(self, SIM, i, j, S):
        
        current_trace = list()
        
        max_value = max(
                    0,
                    SIM[i-1][j-1] + self.evaluate_elements(self.sequence_A[i-1], self.sequence_B[j-1]),
                    SIM[i-1][j  ] + self.gap,
                    SIM[i  ][j-1] + self.gap
        )
        
        if max_value == 0:
            current_trace.append("done")
        
        if max_value == SIM[i-1][j-1] + self.evaluate_elements(self.sequence_A[i-1], self.sequence_B[j-1]):
            current_trace.append("diag")
        
        if max_value == SIM[i-1][j  ] + self.gap:
            current_trace.append("up")
            
        if max_value == SIM[i  ][j-1] + self.gap:
            current_trace.append("left")
        
        return max_value, current_trace, max(S, max_value)
        
    def evaluate_elements(self, element_A, element_B):
       
        if element_A == element_B:
            return self.match
        
        if element_A != element_B:
            return self.mismatch
        
    def global_alignment(self, trace):
        
        m = len(self.sequence_A) + 1
        n = len(self.sequence_B) + 1
        
        direction = trace[m-1][n-1]
        idx       = (m-1, n-1)
        path      = [idx]
        alignment = ("", "")

        while direction[0] != "done":
            direction, idx, alignment = self.move(trace, direction[0], idx, alignment)
            path.append(idx)

        return path, alignment[0][::-1], alignment[1][::-1]
    
    def local_alignment(self, trace, starting_idx): 
        
        m,n       = starting_idx
        direction = trace[m][n]
        idx       = (m, n)
        path      = [idx]
        alignment = ("", "")

        while direction[0] != "done":
            direction, idx, alignment = self.move(trace, direction[0], idx, alignment)
            path.append(idx)

        return path, alignment[0][::-1], alignment[1][::-1]
    
    def move(self, trace, direction, idx, alignment):        
        
        m, n                     = idx
        alignment_A, alignment_B = alignment

        if direction == "diag":
            m -= 1
            n -= 1
            
            alignment_A += self.sequence_A[m]
            alignment_B += self.sequence_B[n]
            
            return trace[m][n], (m, n), (alignment_A, alignment_B)

        if direction == "left":
            n -= 1
            
            alignment_A += "_"
            alignment_B += self.sequence_B[n]
            
            return trace[m][n], (m, n), (alignment_A, alignment_B)

        if direction == "up":
            m -= 1
            
            alignment_A += self.sequence_A[m]
            alignment_B += "_"
            
            return trace[m][n], (m, n), (alignment_A, alignment_B)

In [3]:
def pprint(matrix):
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print ('\n'.join(table))